# Multi-objective Optimization with PARETO

The purpose of this Jupyter notebook is to show how to apply the epsilon constraint method for multi-objective optimization (MOO) to a PARETO model. We will explore the tradeoff between the competing objectives of minimizing the cost of a produced water network and the subsurface risk incurred by operating the network.

We begin by importing all necessary modules and loading the required data (for this demonstration, we will make use of PARETO's strategic toy case study):

In [ ]:
#####################################################################################################
# PARETO was produced under the DOE Produced Water Application for Beneficial Reuse Environmental
# Impact and Treatment Optimization (PARETO), and is copyright (c) 2021-2024 by the software owners:
# The Regents of the University of California, through Lawrence Berkeley National Laboratory, et al.
# All rights reserved.
#
# NOTICE. This Software was developed under funding from the U.S. Department of Energy and the U.S.
# Government consequently retains certain rights. As such, the U.S. Government has been granted for
# itself and others acting on its behalf a paid-up, nonexclusive, irrevocable, worldwide license in
# the Software to reproduce, distribute copies to the public, prepare derivative works, and perform
# publicly and display publicly, and to permit others to do so.
#####################################################################################################

from pareto.strategic_water_management.strategic_produced_water_optimization import (
    WaterQuality,
    create_model,
    Objectives,
    solve_model,
    PipelineCost,
    PipelineCapacity,
    Hydraulics,
    RemovalEfficiencyMethod,
    InfrastructureTiming,
    SubsurfaceRisk,
)
from pareto.utilities.get_data import get_data
from pareto.utilities.results import is_feasible, nostdout
from pyomo.environ import Constraint, Param, value, TerminationCondition, SolverStatus
from importlib import resources

# This emulates what the pyomo command-line tools does
# Tabs in the input Excel spreadsheet
set_list = [
    "ProductionPads",
    "CompletionsPads",
    "SWDSites",
    "ExternalWaterSources",
    "WaterQualityComponents",
    "StorageSites",
    "TreatmentSites",
    "ReuseOptions",
    "NetworkNodes",
    "PipelineDiameters",
    "StorageCapacities",
    "InjectionCapacities",
    "TreatmentCapacities",
    "TreatmentTechnologies",
]
parameter_list = [
    "Units",
    "PNA",
    "CNA",
    "CCA",
    "NNA",
    "NCA",
    "NKA",
    "NRA",
    "NSA",
    "FCA",
    "RCA",
    "RNA",
    "RSA",
    "SCA",
    "SNA",
    "ROA",
    "RKA",
    "SOA",
    "NOA",
    "PCT",
    "PKT",
    "FCT",
    "CST",
    "CCT",
    "CKT",
    "RST",
    "ROT",
    "SOT",
    "RKT",
    "Elevation",
    "CompletionsPadOutsideSystem",
    "DesalinationTechnologies",
    "DesalinationSites",
    "BeneficialReuseCost",
    "BeneficialReuseCredit",
    "TruckingTime",
    "CompletionsDemand",
    "PadRates",
    "FlowbackRates",
    "WellPressure",
    "NodeCapacities",
    "InitialPipelineCapacity",
    "InitialPipelineDiameters",
    "InitialDisposalCapacity",
    "InitialTreatmentCapacity",
    "ReuseMinimum",
    "ReuseCapacity",
    "ExtWaterSourcingAvailability",
    "PadOffloadingCapacity",
    "CompletionsPadStorage",
    "DisposalOperationalCost",
    "TreatmentOperationalCost",
    "ReuseOperationalCost",
    "PipelineOperationalCost",
    "ExternalSourcingCost",
    "TruckingHourlyCost",
    "PipelineDiameterValues",
    "DisposalCapacityIncrements",
    "InitialStorageCapacity",
    "StorageCapacityIncrements",
    "TreatmentCapacityIncrements",
    "TreatmentEfficiency",
    "RemovalEfficiency",
    "DisposalExpansionCost",
    "StorageExpansionCost",
    "TreatmentExpansionCost",
    "PipelineCapexDistanceBased",
    "PipelineCapexCapacityBased",
    "PipelineCapacityIncrements",
    "PipelineExpansionDistance",
    "Hydraulics",
    "Economics",
    "ExternalWaterQuality",
    "PadWaterQuality",
    "StorageInitialWaterQuality",
    "PadStorageInitialWaterQuality",
    "DisposalOperatingCapacity",
    "TreatmentExpansionLeadTime",
    "DisposalExpansionLeadTime",
    "StorageExpansionLeadTime",
    "PipelineExpansionLeadTime_Dist",
    "PipelineExpansionLeadTime_Capac",
    "SWDDeep",
    "SWDAveragePressure",
    "SWDProxPAWell",
    "SWDProxInactiveWell",
    "SWDProxEQ",
    "SWDProxFault",
    "SWDProxHpOrLpWell",
    "SWDRiskFactors",
]

with resources.path(
    "pareto.case_studies",
    "strategic_toy_case_study.xlsx",
) as fpath:
    [df_sets, df_parameters] = get_data(fpath, set_list, parameter_list)

## Epsilon constraint method

The first MOO algorithm we will implement is known as the epsilon constraint method. To proceed, we must create two different copies of the PARETO model. These two models are identical, except they have different objective functions.

In [ ]:
# Model settings
default = {
    "objective": Objectives.cost,
    "pipeline_cost": PipelineCost.distance_based,
    "pipeline_capacity": PipelineCapacity.input,
    "hydraulics": Hydraulics.false,
    "node_capacity": True,
    "water_quality": WaterQuality.false,
    "removal_efficiency_method": RemovalEfficiencyMethod.concentration_based,
    "infrastructure_timing": InfrastructureTiming.false,
    "subsurface_risk": SubsurfaceRisk.exclude_risky_wells,
}

# Create minimum cost model
model_min_cost = create_model(
    df_sets,
    df_parameters,
    default=default,
)

# Change the setting for the objective function from cost to subsurface risk
default["objective"] = Objectives.subsurface_risk

# Create minimum subsurface risk model
model_min_risk = create_model(
    df_sets,
    df_parameters,
    default=default,
)

Next, we set the solver options, solve both of the models, and check that the solutions are feasible and optimal.

In [ ]:
options = {
    "deactivate_slacks": True,
    "scale_model": False,
    "scaling_factor": 1000,
    "running_time": 200,
    "gap": 0,
}

results_min_cost = solve_model(model=model_min_cost, options=options)
results_min_risk = solve_model(model=model_min_risk, options=options)

assert results_min_cost.solver.termination_condition == TerminationCondition.optimal
assert results_min_cost.solver.status == SolverStatus.ok
assert results_min_risk.solver.termination_condition == TerminationCondition.optimal
assert results_min_risk.solver.status == SolverStatus.ok

with nostdout():
    feasibility_status_min_cost = is_feasible(model_min_cost)
    feasibility_status_min_risk = is_feasible(model_min_risk)

assert feasibility_status_min_cost
assert feasibility_status_min_risk

We investigate the value of each objective function in both models: 

In [ ]:
min_cost_obj = model_min_cost.v_Z.value
min_cost_risk = value(model_min_cost.e_SubsurfaceRisk)
min_risk_obj = model_min_risk.v_Z.value
min_risk_cost = value(model_min_risk.e_Cost)

print("Minimum cost model")
print("------------------")
print(f"Minimum cost achievable: {min_cost_obj} kUSD")
print(f"Risk incurred with minimum cost: {min_cost_risk} kbbl/week")

print("\nMinimum risk model")
print("------------------")
print(f"Minimum risk achievable: {min_risk_obj} kbbl/week")
print(f"Cost incurred with minimum risk: {min_risk_cost} kUSD")

We see that when minimizing subsurface risk, it is possible to achieve an objective function value of zero, but it comes at a prohibitively high cost. Note, however, that it is possible to achieve zero total risk for a much lower cost. The reason the cost reported above is so high is that the optimizer arbitrarily chooses to build many pieces of infrastructure which are not needed, since there is no penalty for doing so.

Note that is not necessarily feasible to achieve zero overall risk for every produced water network model, but it happens to be feasible with this particular model. 

Next, we implement the epsilon constraint method (and in so doing, demonstrate that it is possible to achieve zero risk with a much lower cost than above). We proceed by adding a constraint to the minimum cost optimization model to constrain the subsurface risk function to a specific value:

In [ ]:
model_min_cost.p_riskVal = Param(mutable=True, initialize=400)
model_min_cost.risk_constraint = Constraint(
    expr=model_min_cost.e_SubsurfaceRisk == model_min_cost.p_riskVal
)

The epsilon constraint method amounts to solving the optimization problem for different values of this constraint:

In [ ]:
risk_vals = list(range(0, 400, 20))
cost_vals = list()

for r in risk_vals:
    print("*" * 60)
    print(f"Solving minimum cost model constraining subsurface risk to {r}")
    print("*" * 60)
    model_min_cost.p_riskVal = r
    results_min_cost = solve_model(model=model_min_cost, options=options)

    assert results_min_cost.solver.termination_condition == TerminationCondition.optimal
    assert results_min_cost.solver.status == SolverStatus.ok

    with nostdout():
        feasibility_status_min_cost = is_feasible(model_min_cost)

    assert feasibility_status_min_cost

    cost_vals.append(model_min_cost.v_Z.value)

risk_vals.append(min_cost_risk)
cost_vals.append(min_cost_obj)

We can plot the results as a Pareto curve (note that the concept of Pareto optimality inspired the name for Project PARETO):

In [ ]:
# Create plot of Pareto curve
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
scatter = ax.scatter(cost_vals, risk_vals, c="blue")
ax.set_xlabel("Cost [k$]")
ax.set_ylabel("Risk [kbbl/week]")

print(f"Cost values: {cost_vals}")
print(f"Risk values: {risk_vals}")

The Pareto curve shows that compared to the pure minimum cost solution, we can significantly reduce the subsurface risk with nearly negligible increases in the cost.